In [ ]:
import logging
from pathlib import Path
import pandas as pd
import geopandas as gpd
from shapely.geometry import LineString, Point
from shapely.ops import nearest_points
from dotenv import dotenv_values

from generator_drainage_units.utils.general_functions import (
    remove_z_dims,
    snap_unconnected_endpoints_to_endpoint_or_line,
    split_waterways_by_endpoints,
)

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
logging.basicConfig(level=logging.DEBUG)

config = dotenv_values("..\\.env")
base_dir = config["BASE_DIR"]
# case_name = "vallei_en_veluwe"
# case_name = "geerestein"
# case_name = "hattemerbroek"
# case_name = "pangelerbeek"
case_name = "Leuvenumse_beek"

In [ ]:
case_path = Path(base_dir, case_name)

In [ ]:
hydroobjecten = gpd.read_file(Path(case_path, "0_basisdata", "hydroobjecten.gpkg"))


def preprocess_hydroobjecten(hydroobjecten):
    # Setup hydroobjecten correctly
    hydroobjecten = hydroobjecten.rename(columns={"CODE": "code"})
    hydroobjecten = hydroobjecten.to_crs(28992)
    hydroobjecten = remove_z_dims(hydroobjecten)

    # Snap hydroobjecten
    hydroobjecten = snap_unconnected_endpoints_to_endpoint_or_line(
        hydroobjecten, snapping_distance=0.05
    )

    # Split_hydroobjecten
    hydroobjecten = hydroobjecten.rename(columns={"code": "CODE"})
    hydroobjecten = split_waterways_by_endpoints(hydroobjecten, hydroobjecten)

    hydroobjecten.to_file(Path(case_path, "1_tussenresultaat", "test.gpkg"))

    return hydroobjecten

In [ ]:
hydroobjecten = preprocess_hydroobjecten(hydroobjecten)